# Homework 3 Final Models (CIFAR-10)

Adam Kiehl  
4/30/23

In [1]:
# import analysis packages
import keras
from keras.applications import ResNet50, VGG16
from keras.callbacks import EarlyStopping
from keras.datasets import cifar10
from keras.layers import AveragePooling2D, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, RandomFlip, RandomRotation, RandomZoom
from keras.regularizers import l2
from keras import models
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# import dataset
(imgTrain, lblTrain), (imgTest, lblTest) = cifar10.load_data()

# scale image data
imgTrain = imgTrain.astype('float32') / 255
imgTest = imgTest.astype('float32') / 255

# factor categorical response
lblTrain = to_categorical(lblTrain)

In [3]:
# set random seed
np.random.seed(4252023)
tf.random.set_seed(4252023)

# define model architecture
model1 = models.Sequential([
    Conv2D(128, (3, 3), padding = 'same', activation = 'relu', input_shape = (32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), padding = 'same', activation = 'relu'),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), padding = 'same', activation = 'relu'),
    Flatten(),
    Dense(64, activation = 'relu'),
    Dense(10, activation = 'softmax')
])

# compile model
model1.compile(optimizer = 'rmsprop',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

# model summary
model1.summary()

# number of epochs
EPOCHS = 50

# early stopping criteria
earlyStop = EarlyStopping(monitor = 'loss', mode = 'min', verbose = 1, patience = 3)

# train model
trained = model1.fit(imgTrain, 
                     lblTrain, 
                     epochs = EPOCHS, 
                     batch_size = 32, 
                     callbacks = earlyStop,
                     verbose = 2)

# predict on test set
pred1 = model1.predict(imgTest)

# prediction accuracy
display(f"Validation accuracy: {round(np.mean(np.argmax(pred1, axis = 1).reshape(10000, 1) == lblTest) * 100, 2)}%")

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 128)       3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 128)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 128)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 32)          36896     
                                              

2023-04-30 11:13:26.178850: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1563/1563 - 29s - loss: 1.4566 - accuracy: 0.4780 - 29s/epoch - 18ms/step
Epoch 2/50
1563/1563 - 27s - loss: 1.0122 - accuracy: 0.6515 - 27s/epoch - 17ms/step
Epoch 3/50
1563/1563 - 27s - loss: 0.8582 - accuracy: 0.7020 - 27s/epoch - 17ms/step
Epoch 4/50
1563/1563 - 27s - loss: 0.7636 - accuracy: 0.7384 - 27s/epoch - 17ms/step
Epoch 5/50
1563/1563 - 27s - loss: 0.6919 - accuracy: 0.7635 - 27s/epoch - 17ms/step
Epoch 6/50
1563/1563 - 27s - loss: 0.6347 - accuracy: 0.7836 - 27s/epoch - 17ms/step
Epoch 7/50
1563/1563 - 27s - loss: 0.5880 - accuracy: 0.7997 - 27s/epoch - 17ms/step
Epoch 8/50
1563/1563 - 27s - loss: 0.5483 - accuracy: 0.8125 - 27s/epoch - 17ms/step
Epoch 9/50
1563/1563 - 27s - loss: 0.5187 - accuracy: 0.8246 - 27s/epoch - 17ms/step
Epoch 10/50
1563/1563 - 27s - loss: 0.4923 - accuracy: 0.8350 - 27s/epoch - 17ms/step
Epoch 11/50
1563/1563 - 27s - loss: 0.4736 - accuracy: 0.8431 - 27s/epoch - 17ms/step
Epoch 12/50
1563/1563 - 27s - loss: 0.4542 - accuracy: 0.8515 - 27s/epoch 

'Validation accuracuy: 71.78%'

In [16]:
# create submission data frame
submission = pd.DataFrame({'id': range(1, len(pred1) + 1), 'class': np.argmax(pred1, axis = 1).reshape(len(pred1), )})

# export submission
submission.to_csv('./submission1.csv', index = False)

In [4]:
# set trainable layers
VGGBase = VGG16(weights = 'imagenet', include_top = False, input_shape = (32, 32, 3))
VGGBase.trainable = True
for layer in VGGBase.layers:
    if layer.name in ['block2_conv1', 'block3_conv1']:
        layer.trainable = True
    else:
        layer.trainable = False

# define model architecture
model2 = models.Sequential([
    VGGBase,
    Flatten(),
    Dense(64, activation = 'relu'),
    Dense(10, activation = 'softmax')
])

# compile model
model2.compile(optimizer = 'rmsprop',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

# model summary
model2.summary()

# number of epochs
EPOCHS = 50

# early stopping criteria
earlyStop = EarlyStopping(monitor = 'loss', mode = 'min', verbose = 1, patience = 3)

# train model
trained = model2.fit(imgTrain, 
                     lblTrain, 
                     epochs = EPOCHS, 
                     batch_size = 32, 
                     callbacks = earlyStop,
                     verbose = 2)

# predict on test set
pred2 = model2.predict(imgTest)

# prediction accuracy
display(f"Validation accuracy: {round(np.mean(np.argmax(pred2, axis = 1).reshape(10000, 1) == lblTest) * 100, 2)}%")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                32832     
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 14,748,170
Trainable params: 402,506
Non-trainable params: 14,345,664
_________________________________________________________________
Epoch 1/50
1563/1563 - 44s - loss: 1.3419 - accuracy: 0.5138 - 44s/epoch - 28ms/step
Epoch 2/50
1563/1563 - 43s - loss: 0.9630 - accuracy: 0.6597 - 43s/epoch - 28ms/ste

'Validation accuracuy: 75.87%'

In [17]:
# create submission data frame
submission = pd.DataFrame({'id': range(1, len(pred2) + 1), 'class': np.argmax(pred2, axis = 1).reshape(len(pred2), )})

# export submission
submission.to_csv('./submission2.csv', index = False)

In [3]:
# set trainable layers
VGGBase = VGG16(weights = 'imagenet', include_top = False, input_shape = (32, 32, 3))
VGGBase.trainable = True
for layer in VGGBase.layers:
    if layer.name in ['block2_conv1', 'block3_conv1']:
        layer.trainable = True
    else:
        layer.trainable = False

# define model hyperparameters
PENALTY = 0.005
RATE = 0.1

# define model architecture
model3 = models.Sequential([
    VGGBase,
    Flatten(),
    Dense(128, activation = 'relu', kernel_regularizer = l2(PENALTY)),
    Dropout(RATE),
    Dense(64, activation = 'relu', kernel_regularizer = l2(PENALTY)),
    Dense(10, activation = 'softmax')
])

# compile model
model3.compile(optimizer = 'rmsprop',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

# model summary
model3.summary()

# number of epochs
EPOCHS = 100

# early stopping criteria
earlyStop = EarlyStopping(monitor = 'loss', mode = 'min', verbose = 1, patience = 3)

# train model
trained = model3.fit(imgTrain, 
                     lblTrain, 
                     epochs = EPOCHS, 
                     batch_size = 32, 
                     callbacks = earlyStop,
                     verbose = 2)

# predict on test set
pred3 = model3.predict(imgTest)

# prediction accuracy
display(f"Validation accuracy: {round(np.mean(np.argmax(pred3, axis = 1).reshape(10000, 1) == lblTest) * 100, 2)}%")

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                              

2023-04-30 13:10:29.208332: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1563/1563 - 47s - loss: 1.6251 - accuracy: 0.4576 - 47s/epoch - 30ms/step
Epoch 2/100
1563/1563 - 45s - loss: 1.1629 - accuracy: 0.6153 - 45s/epoch - 29ms/step
Epoch 3/100
1563/1563 - 45s - loss: 1.0232 - accuracy: 0.6639 - 45s/epoch - 29ms/step
Epoch 4/100
1563/1563 - 45s - loss: 0.9442 - accuracy: 0.6920 - 45s/epoch - 29ms/step
Epoch 5/100
1563/1563 - 45s - loss: 0.8816 - accuracy: 0.7134 - 45s/epoch - 29ms/step
Epoch 6/100
1563/1563 - 45s - loss: 0.8318 - accuracy: 0.7305 - 45s/epoch - 29ms/step
Epoch 7/100
1563/1563 - 45s - loss: 0.7947 - accuracy: 0.7454 - 45s/epoch - 29ms/step
Epoch 8/100
1563/1563 - 46s - loss: 0.7608 - accuracy: 0.7549 - 46s/epoch - 30ms/step
Epoch 9/100
1563/1563 - 45s - loss: 0.7224 - accuracy: 0.7697 - 45s/epoch - 29ms/step
Epoch 10/100
1563/1563 - 45s - loss: 0.6925 - accuracy: 0.7787 - 45s/epoch - 29ms/step
Epoch 11/100
1563/1563 - 45s - loss: 0.6689 - accuracy: 0.7879 - 45s/epoch - 29ms/step
Epoch 12/100
1563/1563 - 45s - loss: 0.6459 - accuracy: 0.7962 -

'Validation accuracy: 78.09%'

In [ ]:
# create submission data frame
submission = pd.DataFrame({'id': range(1, len(pred3) + 1), 'class': np.argmax(pred3, axis = 1).reshape(len(pred3), )})

# export submission
submission.to_csv('./submission3.csv', index = False)